In [1]:
import tensorflow as tf

from tensorflow import keras

import numpy as np

import pandas as pd

import os

import sys

import matplotlib.pyplot as plt

%matplotlib inline

#!{sys.executable} -m pip install opencv-python

import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

Using TensorFlow backend.


In [2]:
test_folder = '../data/test'
train_folder = '../data/train'
val_folder = '../data/val'

In [117]:
# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64,64), batch_size = 627) 

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder, 
        target_size=(64,64), batch_size = 19)

# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_folder, 
        target_size=(64,64), batch_size=5219)

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [99]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [100]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 5216
Number of testing samples: 624
Number of validation samples: 16
train_images shape: (5216, 64, 64, 3)
train_labels shape: (5216, 2)
test_images shape: (624, 64, 64, 3)
test_labels shape: (624, 2)
val_images shape: (16, 64, 64, 3)
val_labels shape: (16, 2)


In [101]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 12288)
(624, 12288)
(16, 12288)


In [102]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

# Modeling

## Baseline fully-connected

In [105]:
from keras import models
from keras import layers
import keras_metrics
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [107]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                    validation_data=(val_img, val_y),
                    verbose=1)

Train on 5216 samples, validate on 16 samples
Epoch 1/50
5216/5216 [==============================] - ETA: 0s - loss: 0.1496 - acc: 0.940 - 1s 194us/step - loss: 0.1481 - acc: 0.9408 - val_loss: 0.7925 - val_acc: 0.6875
Epoch 2/50
5216/5216 [==============================] - 1s 109us/step - loss: 0.1321 - acc: 0.9500 - val_loss: 0.9314 - val_acc: 0.6250
Epoch 3/50
5216/5216 [==============================] - 1s 105us/step - loss: 0.1514 - acc: 0.9404 - val_loss: 0.4808 - val_acc: 0.8125
Epoch 4/50
5216/5216 [==============================] - 1s 103us/step - loss: 0.1268 - acc: 0.9523 - val_loss: 0.5393 - val_acc: 0.8125
Epoch 5/50
5216/5216 [==============================] - 1s 106us/step - loss: 0.1271 - acc: 0.9473 - val_loss: 0.3268 - val_acc: 0.8750
Epoch 6/50
5216/5216 [==============================] - 1s 104us/step - loss: 0.1235 - acc: 0.9551 - val_loss: 0.7370 - val_acc: 0.6875
Epoch 7/50
5216/5216 [==============================] - 1s 103us/step - loss: 0.1279 - acc: 0.9496 -

In [108]:
results_train = model.evaluate(train_img, train_y)

5216/5216 [==============================] - 0s 53us/step


In [109]:
results_test = model.evaluate(test_img, test_y)

624/624 [==============================] - 0s 59us/step


In [110]:
results_train

[0.07977211311390978, 0.9702837423312883]

In [111]:
results_test

[0.7531664157525088, 0.7916666666666666]

## CNN

In [115]:
import tensorflow as tf

In [116]:
print(tf.__version__)

1.6.0


In [112]:
import keras_metrics
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(64,64,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dense(4, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [113]:
history = model.fit(train_images,
                    train_y,
                    epochs=1,
                    batch_size=32,
                    validation_data=(val_images, val_y),
                    verbose=1)

ValueError: Error when checking target: expected dense_37 to have 4 dimensions, but got array with shape (5216, 1)

In [73]:
results_train = model.evaluate(train_images, train_y)

5216/5216 [==============================] - 126s 24ms/step


In [74]:
results_test = model.evaluate(test_images, test_y)

624/624 [==============================] - 16s 26ms/step


In [75]:
results_train

[0.28860043190373963, 0.8989266546602033, 0.7494407158277822]

In [76]:
results_test

[0.43435644950622165, 0.9133858260524521, 0.4957264955146468]

In [95]:
! tf --version

/bin/sh: tf: command not found


In [83]:
import keras_metrics
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(256,256,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [84]:
history = model.fit(train_images,
                    train_y,
                    epochs=1,
                    batch_size=32,
                    validation_data=(val_images, val_y),
                    verbose=1)

Train on 5216 samples, validate on 16 samples
Epoch 1/1
 256/5216 [>.............................] - ETA: 5:02 - loss: 0.6088 - precision: 0.0000e+00 - recall: 0.0000e+00


KeyboardInterrupt



In [ ]:
results_train = model.evaluate(train_images, train_y)

In [ ]:
results_test = model.evaluate(test_images, test_y)

In [ ]:
results_train

In [ ]:
results_test

In [ ]:
model.summary()